In [ ]:
!! pip install open-metric-learning
!! pip install pytorch-metric-learning
!! pip install pillow==9.0.0

['Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/',
 'Collecting pillow==9.0.0',
 '  Downloading Pillow-9.0.0-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (4.3 MB)',
 '\x1b[?25l     \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b0.0/4.3 MB\x1b \x1b?\x1b eta \x1b-:--:--\x1b',
 '\x1b[2K     \x1b━━━━━━━━━━━━━━━━━━\x1b\x1b╺\x1b\x1b━━━━━━━━━━━━━━━━━━━━━\x1b \x1b2.0/4.3 MB\x1b \x1b58.1 MB/s\x1b eta \x1b0:00:01\x1b',
 '\x1b[2K     \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b\x1b╸\x1b \x1b4.3/4.3 MB\x1b \x1b81.2 MB/s\x1b eta \x1b0:00:01\x1b',
 '\x1b[2K     \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b4.3/4.3 MB\x1b \x1b52.8 MB/s\x1b eta \x1b0:00:00\x1b',
 '\x1b[?25hInstalling collected packages: pillow',
 '  Attempting uninstall: pillow',
 '    Found existing installation: Pillow 8.4.0',
 '    Uninstalling Pillow-8.4.0:',
 '      Successfully uninstalled Pillow-8.4.0',
 'Successfully installed pillow-9.0.0']

In [ ]:
import torch
from tqdm import tqdm

from oml.datasets.base import DatasetWithLabels
from oml.losses.triplet import TripletLossWithMiner
from oml.miners.inbatch_all_tri import AllTripletsMiner
from oml.models.vit.vit import ViTExtractor
from oml.samplers.balance import BalanceSampler
from oml.datasets.base import DatasetQueryGallery
from oml.metrics.embeddings import EmbeddingMetrics
from oml.utils.dataframe_format import check_retrieval_dataframe_format
from oml.utils.misc_torch import PCA
from oml.inference.flat import inference_on_images
from oml.registry.transforms import get_transforms_for_pretrained
from oml.utils.misc_torch import pairwise_dist
from pytorch_metric_learning import losses, distances, reducers, miners
import pandas as pd
from oml.transforms.images.torchvision import get_normalisation_resize_torch
from oml.models.meta.siamese import ConcatSiamese
from oml.miners.pairs import PairsMiner
from oml.inference.flat import inference_on_dataframe
from torch.nn import BCEWithLogitsLoss
from torch.utils.data import DataLoader
from oml.retrieval.postprocessors.pairwise import PairwiseImagesPostprocessor
from pprint import pprint



In [ ]:
%cd '/content/drive/MyDrive/iris'

/content/drive/MyDrive/iris


In [ ]:
import os
from os import walk
import pandas as pd

# Create a sample DataFrame
data = {'label': [],
        'src_file': [],
        'split': [],
        'is_query': [],
        'is_gallery': [],
        'eye': [],
        'ds': [],
        'path': []
        }
df_zenica = pd.DataFrame(data)

# list to store files
res = []

rootdir = 'ZENICA_IRIS'

def listdirs(rootdir):
    for file in os.listdir(rootdir):
        d = os.path.join(rootdir, file)
        if os.path.isdir(d):
            #print(d)
            listdirs(d)
        else:
          res.append(d)



listdirs(rootdir)








In [ ]:
is_query=1
is_gallery=0.0
ds='zenica'

for r in res:
  z,q,label,lr,full,src_file=r.split('/')

  if lr=='right':
    lr='R'
  else:
     lr='L'


  if (is_query==1.0):
     is_query=0.0
     is_gallery=1.0
  else:
     is_query=1.0
     is_gallery=0.0


  new_row = {'label': label,
             'src_file': src_file,
             'split': 'validation',
             'is_query': is_query,
             'is_gallery': is_gallery,
             'eye': lr,
             'ds': ds,
             'path': r
             }

  df_zenica.loc[len(df_zenica)] = new_row



In [ ]:
csvpath="/content/drive/MyDrive/iris/zenica_iris.csv"
#df_zenica.to_csv(csvpath,index=False)

In [ ]:
import pandas as pd

csvpath="/content/drive/MyDrive/iris/zenica_iris.csv"
df_zenica = pd.read_csv(csvpath)

In [ ]:
df_zenica

,label,src_file,split,is_query,is_gallery,eye,ds,path
0,5,05_right_full_8.png,validation,0.0,1.0,R,zenica,ZENICA_IRIS/1920x1080/5/right/full/05_right_fu...
1,5,05_right_full_7.png,validation,1.0,0.0,R,zenica,ZENICA_IRIS/1920x1080/5/right/full/05_right_fu...
2,5,05_right_full_6.png,validation,0.0,1.0,R,zenica,ZENICA_IRIS/1920x1080/5/right/full/05_right_fu...
3,5,05_right_full_5.png,validation,1.0,0.0,R,zenica,ZENICA_IRIS/1920x1080/5/right/full/05_right_fu...
4,5,05_right_full_4.png,validation,0.0,1.0,R,zenica,ZENICA_IRIS/1920x1080/5/right/full/05_right_fu...
...,...,...,...,...,...,...,...,...
155,1,01_right_full_4.png,validation,1.0,0.0,R,zenica,ZENICA_IRIS/1920x1080/1/right/full/01_right_fu...
156,1,01_right_full_5.png,validation,0.0,1.0,R,zenica,ZENICA_IRIS/1920x1080/1/right/full/01_right_fu...
157,1,01_right_full_6.png,validation,1.0,0.0,R,zenica,ZENICA_IRIS/1920x1080/1/right/full/01_right_fu...
158,1,01_right_full_7.png,validation,0.0,1.0,R,zenica,ZENICA_IRIS/1920x1080/1/right/full/01_right_fu...


In [ ]:
%cd /content/drive/MyDrive/iris

/content/drive/MyDrive/iris


In [ ]:
!python3 ./run_iris_detect_knn3.py

init success
Prefix <model.> was removed from the state dict.
[31.78305327664316, 10.199534042785553, 12.036388538421795, 18.66858299954599, 14.215654711169215, 26.291762491364928, 12.200306897915327, 17.461309769521513, 9.010651537235711, 13.007028880300403, 13.55402443443056, 10.212345834640004, 24.219464455187563, 10.980665033486666, 13.18746644508468, 11.51665518098103, 9.989358198687226, 21.544532963950868, 14.442218752501887, 15.88593162160558, 14.614346267325441, 12.237891616244095, 13.724246234033505, 32.52748612351752, 27.65044739621653, 16.057250593761665, 15.358520111477162, 14.914980716724827, 19.78496537111774, 16.75649761741229, 16.752476751063465, 15.471702586742584, 18.396485100760415, 21.2738915267837, 23.329106120527047, 20.21876056558843, 23.693420475484857, 25.286318201422556, 26.7466365803137, 21.227162734198384, 22.886253048790692, 21.87687910455851, 12.1478986516348, 8.590691039385579, 18.456906828284332, 13.173199606254014, 17.150388734376676, 8.313905984243732,

In [ ]:
df_04_right_full_6 = df_zenica[df_zenica['src_file']=='04_right_full_6.png']

In [ ]:
df_04_right_full_6

,label,src_file,split,is_query,is_gallery,eye,ds,path
66,4,04_right_full_6.png,validation,1.0,0.0,R,zenica,ZENICA_IRIS/1920x1080/4/right/full/04_right_fu...


In [ ]:
df_zenica.at[66,'is_query']=1.0
df_zenica.at[66,'is_gallery']=0.0

In [ ]:
df_zenica.iloc[66]['is_query']

1.0

In [ ]:
df04_right_full_4 = df_zenica[df_zenica['src_file']=='04_right_full_4.png']

In [ ]:
df04_right_full_4

,label,src_file,split,is_query,is_gallery,eye,ds,path
68,4,04_right_full_4.png,validation,1.0,0.0,R,zenica,ZENICA_IRIS/1920x1080/4/right/full/04_right_fu...


In [ ]:
df_zenica.at[68,'is_query']=1.0
df_zenica.at[68,'is_gallery']=0.0